215. Kth Largest Element in an Array
Solved
Medium
Topics
Companies
Given an integer array nums and an integer k, return the kth largest element in the array.

Note that it is the kth largest element in the sorted order, not the kth distinct element.

Can you solve it without sorting?

 

Example 1:

Input: nums = [3,2,1,5,6,4], k = 2
Output: 5
Example 2:

Input: nums = [3,2,3,1,2,4,5,5,6], k = 4
Output: 4
 

Constraints:

1 <= k <= nums.length <= 105
-104 <= nums[i] <= 104

Approach 3: Quickselect
Intuition

This is a more advanced/esoteric algorithm. Do not feel discouraged if you are unable to derive it yourself. It is highly unlikely that you would be expected to come up with this solution in an interview without any help from the interviewer.

Quickselect, also known as Hoare's selection algorithm, is an algorithm for finding the kthk^{th}k 
th
  smallest element in an unordered list. It is significant because it has an average runtime of O(n)O(n)O(n).

Quickselect uses the same idea as Quicksort. First, we choose a pivot index. The most common way to choose the pivot is randomly. We partition nums into 3 sections: elements equal to the pivot, elements greater than the pivot, and elements less than the pivot.

Next, we count the elements in each section. Let's denote the sections as follows:

left is the section with elements less than the pivot
mid is the section with elements equal to the pivot
right is the section with elements greater than the pivot
Quickselect is normally used to find the kthk^{th}k 
th
  smallest element, but we want the kthk^{th}k 
th
  largest. To account for this, we will swap what left and right represent - left will be the section with elements greater than the pivot, and right will be the section with elements less than the pivot.

If the number of elements in left is greater than or equal to k, the answer must be in left - any other element would be less than the kthk^{th}k 
th
  largest element. We restart the process in left.



If the number of elements in left and mid is less than k, the answer must be in right - any element in left or mid would not be large enough to be the kthk^{th}k 
th
  largest element. We restart the process in right.



There's one extra step if the answer is in right. When we go to search in right, we are effectively "deleting" the elements in left and mid (since they will never be considered again). Because the elements in left and mid are greater than the answer, deleting them means we must shift k. Let's say we're looking for the 8th8^{th}8 
th
  greatest element, but then we delete the 4 greatest elements. In the remaining data, we would be looking for the 4th4^{th}4 
th
  greatest element, not the 8th. Therefore, we need to subtract the length of left and mid from k when we search for right.

We don't need to modify k when we search in left because when we search in left, we delete elements smaller than the answer, which doesn't affect k.



If the answer is in neither left or right, it must be in mid. Since mid only has elements equal to the pivot, the pivot must be the answer.

The easiest way to implement this repetitive process is by using recursion.

Algorithm

Note: the implementation we use here is not a standard Quickselect implementation. We will be using slightly more space (still the same complexity), but in exchange, we will be writing significantly less code.

Define a quickSelect function that takes arguments nums and k. This function will return the kthk^{th}k 
th
  greatest element in nums (the nums and k given to it as input, not the original nums and k).

Select a random element as the pivot.
Create left, mid, and right as described above.
If k <= left.length, return quickSelect(left, k).
If left.length + mid.length < k, return quickSelect(right, k - left.length - mid.length).
Otherwise, return pivot.
Call quickSelect with the original nums and k, and return the answer.
Complexity Analysis

Given nnn as the length of nums,

Time complexity: O(n) on average, O(n^2) in the worst case

Each call we make to quickSelect will cost O(n) since we need to iterate over nums to create left, mid, and right. The number of times we call quickSelect is dependent on how the pivots are chosen. The worst pivots to choose are the extreme (greatest/smallest) ones because they reduce our search space by the least amount. Because we are randomly generating pivots, we may end up calling quickSelect O(n) times, leading to a time complexity of O(n^2).

However, the algorithm mathematically almost surely has a linear runtime. For any decent size of nums, the probability of the pivots being chosen in a way that we need to call quickSelect O(n) times is so low that we can ignore it.

On average, the size of nums will decrease by a factor of ~2 on each call. You may think: that means we call quickSelect O(log⁡n)O(\log n)O(logn) times, wouldn't that give us a time complexity of O(n⋅log⁡n)? Well, each successive call to quickSelect would also be on a nums that is a factor of ~2 smaller. This recurrence can be analyzed using the master theorem with a = 1, b = 2, k = 1:

T(n)=T(n2)+O(n)=O(n)

Space complexity: O(n)
We need O(n) space to create left, mid, and right. Other implementations of Quickselect can avoid creating these three in memory, but in the worst-case scenario, those implementations would still require O(n) space for the recursion call stack.


Bonus
When we randomly choose pivots, Quickselect has a worst-case scenario time complexity of O(n^2).
By using the median of medians algorithm, we can improve to a worst-case scenario time complexity of O(n).

This approach is way out of scope for an interview, and practically it isn't even worth implementing because there is a large constant factor. As stated above, the random pivot approach will yield a linear runtime with mathematical certainty, so in all practical scenarios, it is sufficient.
The median of medians approach should only be appreciated for its theoretical beauty. Those who are interested can read more using the link above.



In [ ]:
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        def quick_select(nums, k):
            pivot = random.choice(nums)

            left = [x for x in nums if x > pivot]
            mid = [x for x in nums if x == pivot]
            right = [x for x in nums if x < pivot]

            if k <= len(left):
                return quick_select(left,k)
            elif k > len(left) + len(mid):
                return quick_select(right, k-len(left)-len(mid))
            else:
                return pivot
            
        return quick_select(nums,k)

In [ ]:
class Solution:
    def findKthLargest(self, nums, k):
        def quick_select(nums, k):
            pivot = random.choice(nums)
            left, mid, right = [], [], []

            for num in nums:
                if num > pivot:
                    left.append(num)
                elif num < pivot:
                    right.append(num)
                else:
                    mid.append(num)
            
            if k <= len(left):
                return quick_select(left, k)
            
            if len(left) + len(mid) < k:
                return quick_select(right, k - len(left) - len(mid))
            
            return pivot
        
        return quick_select(nums, k)

Approach 2: Min-Heap
Intuition

A heap is a very powerful data structure that allows us to efficiently find the maximum or minimum value in a dynamic dataset.

If you are not familiar with heaps, we recommend checking out the Heap Explore Card.

The problem is asking for the kthk^{th}k 
th
  largest element. Let's push all the elements onto a min-heap, but pop from the heap when the size exceeds k. When we pop, the smallest element is removed. By limiting the heap's size to k, after handling all elements, the heap will contain exactly the kkk largest elements from the array.



It is impossible for one of the green elements to be popped because that would imply there are at least kkk elements in the array greater than it. This is because we only pop when the heap's size exceeds k, and popping removes the smallest element.

After we handle all the elements, we can just check the top of the heap. Because the heap is holding the kkk largest elements and the top of the heap is the smallest element, the top of the heap would be the kthk^{th}k 
th
  largest element, which is what the problem is asking for.

Algorithm

Initialize a min-heap heap.
Iterate over the input. For each num:
Push num onto the heap.
If the size of heap exceeds k, pop from heap.
Return the top of the heap.
Implementation

Note: C++ std::priority_queue implements a max-heap. To achieve min-heap functionality, we will multiply the values by -1 before pushing them onto the heap.
Complexity Analysis

Given nnn as the length of nums,

Time complexity: O(n⋅log⁡k)

Operations on a heap cost logarithmic time relative to its size. Because our heap is limited to a size of k, operations cost at most O(log⁡k). We iterate over nums, performing one or two heap operations at each iteration.

We iterate nnn times, performing up to log⁡k work at each iteration, giving us a time complexity of O(n⋅log⁡k).

Because k≤n, this is an improvement on the previous approach.

Space complexity: O(k)

The heap uses O(k) space.

In [ ]:
# T O(nlog(k))
# S O(k)
class Solution:
    def findKthLargest(self, nums, k):
        heap = []
        for num in nums:
            heapq.heappush(heap, num)
            if len(heap) > k:
                heapq.heappop(heap)
        
        return heap[0]

In [ ]:
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        heap = []

        for num in nums:
            if len(heap) < k:
                heapq.heappush(heap, num)
            else:
                if num > heap[0]:
                    heapq.heappop(heap)
                    heapq.heappush(heap,num) 
        return heap[0]

Approach 1: Sort
Intuition

Sort the array in descending order and then return the kthk^{th}k 
th
  element. Note that this is the "trivial" approach and if asked this question in an interview, you would be expected to come up with a better solution than this.

Implementation

Note: k is 1-indexed, not 0-indexed. As such, we need to return the element at index k - 1 after sorting, not index k.Complexity Analysis

Given nnn as the length of nums,

Time complexity: O(n⋅log⁡n)O(n \cdot \log n)O(n⋅logn)

Sorting nums requires O(n⋅log⁡n)O(n \cdot \log n)O(n⋅logn) time.

Space Complexity: O(log⁡n)O(\log n)O(logn) or O(n)O(n)O(n)

The space complexity of the sorting algorithm depends on the implementation of each programming language:

In Java, Arrays.sort() for primitives is implemented using a variant of the Quick Sort algorithm, which has a space complexity of O(log⁡n)O(\log n)O(logn)
In C++, the sort() function provided by STL uses a hybrid of Quick Sort, Heap Sort, and Insertion Sort, with a worst-case space complexity of O(log⁡n)O(\log n)O(logn)
In Python, the sort() function is implemented using the Timsort algorithm, which has a worst-case space complexity of O(n)O(n)O(n)

In [ ]:
class Solution:
    def findKthLargest(self, nums, k):
        nums.sort(reverse=True)
        return nums[k - 1]

In [ ]:
class Solution:
    def findKthLargest(self, nums, k):
        nums.sort()
        return nums[len(nums) - k]